In [ ]:
from manim import *
import json
from scipy.interpolate import interp1d

In [ ]:
with open('./ideas/positions.json', 'r') as f:
    path = json.load(f)
# with open('./ideas/positions2.json', 'r') as f:
#     path = json.load(f)
    
path = np.array([p+[0.0] for p in path])*0.018+0.1
f = interp1d(np.linspace(0,1,3001), path, axis=0)

In [ ]:
%%manim DrawPath

class DrawPath(MovingCameraScene):
    def construct(self):
        
        def update_bee(bee, dt):
            new_pos = f(tracker.get_value())
            tsub1 = f(np.maximum(0, tracker2.get_value()))
            
            diff = new_pos-tsub1
            angle = np.arctan2(diff[1], diff[0])
            bee.points = bee.original_points.copy()
                
            bee.rotate(angle-PI/2)
            bee.move_to(new_pos)
            
            tracker3.set_value(tracker2.get_value())
            tracker2.set_value(tracker.get_value())
        
        outbound = ParametricFunction(f, t_range=[0,0.5], color=BLUE)
        inbound = ParametricFunction(f, t_range=[0.5,1], color=ORANGE)
        
        label1 = Text("Outbound").move_to([10,6,0])
        line1 = Line([0,0,0],[1,0,0],color=BLUE,stroke_width=10).next_to(label1,LEFT)
        label2 = Text("Inbound").next_to(label1,DOWN)
        line2 = Line([0,0,0],[1,0,0],color=ORANGE,stroke_width=10).next_to(label2,LEFT)
        
        tracker = ValueTracker(0)
        tracker2 = ValueTracker(0)
        tracker3 = ValueTracker(0)
        
        bee = ImageMobject("./bee_lowres.png").add_updater(update_bee).set_z_index(10)
        bee.height=0.2
        bee.original_points = bee.points.copy()
        nest = ImageMobject("./branch.png").rotate(PI / 4)
        nest.move_to([-0.7,0.1,0])
        nest.height = 2
        
        self.camera.frame.move_to([5,3,0])  
        self.add(nest,tracker,tracker2,tracker3)
        
        self.play(
            Write(label1),
            Write(label2),
            Create(line1),
            Create(line2),
        )
        self.wait(1)
        
        self.add(bee)
        self.play(Create(outbound,rate_func=rate_functions.ease_in_sine),tracker.animate.set_value(0.5),run_time=7)
        self.wait(0.2)
        self.play(Create(inbound),tracker.animate.set_value(1),run_time=6)
        self.wait(3)